##### Master Degree in Computer Science and Data Science for Economics

# Prompt Engineering

### Alfio Ferrara

For an introduction see
> Schulhoff, S., Ilie, M., Balepur, N., Kahadze, K., Liu, A., Si, C., ... & Resnik, P. (2024). The prompt report: A systematic survey of prompting techniques. arXiv preprint arXiv:2406.06608, 5.

## Options to interact with LLMs
-  Cloud APIs (e.g., OpenAI, Anthropic)
    - They are tipically easy-to-use but not always free. Data are processed online in cloud.
- Hugging Face Transformers (or other libraries)
    - Multiple models are available, and provides a complete control over the pipeline. Howevere everything is executed locally and a GPU is highly recommended.
- Runtime interfaces
    - There are multiple options, such as [llama.cpp](https://github.com/ggml-org/llama.cpp), [ollama](https://ollama.com/), [vllm](https://github.com/vllm-project/vllm)
    - They support multiple models and both local and remote execution

### A first example (using huggingface)

Sometimes a authentication token is required. See [https://huggingface.co/settings/tokens](https://huggingface.co/settings/tokens)

In [1]:
import json
from huggingface_hub import login

In [3]:
with open('/Users/Flint/Data/apikeys/keys.json', 'r') as infile:
    token = json.load(infile)['huggingface']

login(token=token)

In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

torch.set_default_device("mps")

model_id = "HuggingFaceH4/zephyr-7b-alpha"

print("Loading model... This may take a few minutes the first time.")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32)  # MPS does not support float16

prompt = "<|system|>You are a helpful assistant.<|user|>What is prompt engineering?<|assistant|>"

inputs = tokenizer(prompt, return_tensors="pt")
inputs = {k: v.to("mps") for k, v in inputs.items()}  # Sposta tutto su MPS

print("Generating...")
outputs = model.generate(**inputs, max_new_tokens=200)

response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("\nZephyr's response:\n")
print(response)


Loading model... This may take a few minutes the first time.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generating...

Zephyr's response:

<|system|>You are a helpful assistant.<|user|>What is prompt engineering?<|assistant|>Prompt engineering is the process of designing and optimizing input prompts for natural language processing (NLP) models to improve their performance and accuracy. This involves selecting the right language, syntax, and structure for the prompts to ensure that the model can understand and respond appropriately to the input. Prompt engineering can also involve the use of pre-trained language models, such as GPT-3, to generate more complex and nuanced prompts that can better capture the intended meaning of the input. Overall, prompt engineering is a critical component of NLP development and can significantly improve the performance and usability of NLP models in various applications.


## Using APIs

In [5]:
from openai import OpenAI

with open('/Users/Flint/Data/apikeys/keys.json', 'r') as infile:
    apikey = json.load(infile)['openai']

client = OpenAI(api_key=apikey)

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is prompt engineering in simple terms?"}
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
    max_tokens=300,
    temperature=0.7
)

print(response.choices[0].message.content)

Prompt engineering is the process of designing and creating effective prompts or cues to encourage specific behaviors or actions. It involves strategically presenting information or stimuli to guide individuals towards a desired outcome or response. This could be used in various contexts such as marketing, education, or behavior change interventions.


## Instruction + context + output format

In [6]:
def askgpt(messages, temperature=0.7):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature)
    return response.choices[0].message.content

**Generic and non-informative prompt**

In [7]:
messages = [
    {"role": "user", "content": "Write a recipe."}
]
answer = askgpt(messages)

print(answer)

Classic Chocolate Chip Cookies

Ingredients:
- 1 cup unsalted butter, softened
- 1 cup granulated sugar
- 1 cup brown sugar
- 2 large eggs
- 1 tsp vanilla extract
- 3 cups all-purpose flour
- 1 tsp baking soda
- 1/2 tsp salt
- 2 cups semi-sweet chocolate chips

Instructions:
1. Preheat oven to 350°F (180°C) and line a baking sheet with parchment paper.
2. In a large mixing bowl, cream together the butter, granulated sugar, and brown sugar until light and fluffy.
3. Add in the eggs one at a time, beating well after each addition. Stir in the vanilla extract.
4. In a separate bowl, whisk together the flour, baking soda, and salt. Gradually add the dry ingredients to the wet ingredients, mixing until just combined.
5. Fold in the chocolate chips until evenly distributed throughout the dough.
6. Using a cookie scoop or spoon, drop rounded tablespoons of dough onto the prepared baking sheet, spacing them about 2 inches apart.
7. Bake in the preheated oven for 10-12 minutes, or until the edg

**Instructions**

Let's add some more instructions on the kind of language we want to be used. Still unclear what kind of recipe we want to get and how formatted.

In [8]:
messages = [
    {"role": "system", "content": "You are a professional chef."},
    {"role": "user", "content": "Write a recipe."}
]

answer = askgpt(messages)
print(answer)

Sure, here's a recipe for a delicious and comforting Chicken Alfredo Pasta:

Ingredients:
- 1 lb fettuccine pasta
- 2 boneless, skinless chicken breasts
- 2 tbsp olive oil
- 3 cloves garlic, minced
- 1 cup heavy cream
- 1 cup grated Parmesan cheese
- 1/2 cup unsalted butter
- Salt and pepper to taste
- Fresh parsley, chopped (for garnish)

Instructions:
1. Cook the fettuccine pasta according to package instructions until al dente. Drain and set aside.
2. Season the chicken breasts with salt and pepper. In a large skillet, heat 1 tbsp of olive oil over medium-high heat. Add the chicken breasts and cook for about 6-7 minutes per side, or until cooked through. Remove from the skillet and let it rest for a few minutes before slicing into strips.
3. In the same skillet, add the remaining tbsp of olive oil and minced garlic. Cook for about 1 minute, until fragrant.
4. Add the heavy cream and butter to the skillet, stirring until the butter is melted and the mixture is well combined.
5. Gradu

**Instructions + Context**

We want to add someting more about the kind of recipe we want to obtain

In [9]:
messages = [
    {"role": "system", "content": "You are writing recipes for a magazine that publishes easy-to-do recipes with few ingredients."},
    {"role": "user", "content": "Write a chinese recipe that an non experienced teenager can follow with easy-to-find ingredients."}
]

answer = askgpt(messages)
print(answer)

**Simple Sweet and Sour Chicken**

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1/2 cup cornstarch
- 1/4 cup vegetable oil
- 1/2 cup ketchup
- 1/4 cup white vinegar
- 1/4 cup sugar
- 1/4 cup soy sauce
- 1/4 cup water
- 1 bell pepper, cut into chunks
- 1 small onion, cut into chunks
- Cooked rice, for serving

Instructions:
1. In a bowl, toss the chicken pieces with cornstarch until well coated.
2. In a large skillet or wok, heat the vegetable oil over medium-high heat.
3. Add the chicken pieces to the skillet and cook until golden brown and cooked through, about 5-7 minutes. Remove the chicken from the skillet and set aside.
4. In the same skillet, add the bell pepper and onion and cook for 2-3 minutes until slightly softened.
5. In a small bowl, whisk together the ketchup, vinegar, sugar, soy sauce, and water. Pour the sauce into the skillet with the vegetables.
6. Add the cooked chicken back to the skillet and stir to coat everything in the sau

**Instructions + Context + output format**

We try to control the output

In [10]:
messages = [
    {"role": "system", "content": "You are writing recipes for a magazine that publishes easy-to-do recipes with few ingredients."},
    {"role": "user", "content": """
    Write a chinese recipe that an non experienced teenager can follow with easy-to-find ingredients.
    Provide the answer in json format like this: {"ingredients": [list of ingredients]}.
    Do not add anything but the ingredients. No title, no description and no comments!
    """}
]

answer = askgpt(messages)
print(answer)

{
    "ingredients": [
        "1 pound boneless, skinless chicken breasts",
        "2 tablespoons soy sauce",
        "1 tablespoon honey",
        "1 tablespoon vegetable oil",
        "2 cloves garlic, minced",
        "1 teaspoon ginger, minced",
        "1 bell pepper, sliced",
        "1 cup broccoli florets",
        "Cooked rice, for serving"
    ]
}


**Note about formatting**

This kind of formatting still requires parsing the json structure. With some libraries, like `llama.cpp` we can define the output using a `grammar`.

See more information on [GGML](https://github.com/ggml-org/llama.cpp/blob/master/grammars/README.md) website.

## Zero-shot, one-show, few-shot

1. **Zero-shot prompting**

    You ask the model to perform a task without showing any examples. It relies only on the instruction and prior training.

2. **One-shot prompting**

    You provide a single example of the task you want done, to guide the model.

3. **Few-shot prompting**

    You provide a few (2–5) examples to demonstrate the pattern or structure you're expecting.

#### Zero-shot

In [11]:
messages = [
    {"role": "system", "content": "You are a professional chef."},
    {"role": "user", "content": "Provide the ingredients of an italian recipe formatted as a simple list"}
]

answer = askgpt(messages)
print(answer)

Sure! Here are the ingredients for a classic Italian spaghetti aglio e olio:

- 1 pound spaghetti
- 4 cloves garlic, thinly sliced
- 1/2 cup extra virgin olive oil
- 1/2 teaspoon red pepper flakes
- Salt and pepper to taste
- Fresh parsley, chopped (for garnish)
- Grated Parmesan cheese (optional)


#### One- and few-shot 

In [14]:
messages = [
    {"role": "system", "content": "You are a professional chef."},
    {"role": "user", "content": """Provide the ingredients of an italian recipe formatted as a simple list
    An example is: "Title: Pasta with tomato and onions"
    "Ingredients": [pasta, olive oil, tomato sauce, onions]
     I want just the title and the ingredients and not the quantities!
    """}
]

answer = askgpt(messages)
print(answer)

Title: Classic Margherita Pizza
Ingredients: [pizza dough, San Marzano tomato sauce, fresh mozzarella cheese, fresh basil leaves, extra-virgin olive oil, salt]


## Prompt chain-of-thought

In [15]:
messages = [
    {"role": "system", "content": "You are a cousine espert."},
    {"role": "user", "content": """
    Provide a review on the pro and cons of the Italian cousine. Follow this scheme:
    1) Introduce the Italian cousine in a single sentence; 2) Summarize the main three pros; 3) summarize the main three cons;
    4) Conclude with an educated suggestion
    """}
]

answer = askgpt(messages)
print(answer)

1) Italian cuisine is renowned for its simple yet flavorful dishes that celebrate fresh ingredients and rich culinary traditions.

2) Pros:
   a) Fresh Ingredients: Italian cuisine relies heavily on fresh, seasonal ingredients such as tomatoes, olive oil, and herbs, resulting in vibrant and delicious dishes.
   b) Diverse Regional Flavors: Italy's diverse regions offer a wide range of culinary specialties, from the hearty dishes of the north to the seafood-focused cuisine of the south, providing a rich tapestry of flavors to explore.
   c) Emphasis on Tradition: Italian cooking is deeply rooted in tradition, with recipes often passed down through generations, preserving authentic flavors and techniques.

3) Cons:
   a) Heavy on Carbs: Italian cuisine is known for its pasta, pizza, and bread, which can be high in carbohydrates and calories, potentially leading to weight gain if consumed excessively.
   b) Limited Vegetarian Options: While Italy offers a variety of meat and seafood dishe

### Prompt with constraints concerning style, lenght, format, etc.

In [16]:
messages = [
    {"role": "system", "content": "You are a cousine espert."},
    {"role": "user", "content": """
    Write a short Italian recipe in the style of Lord Byron
    """}
]

answer = askgpt(messages)
print(answer)

Oh, the blissful art of Italian cuisine,
In the charming hills of Tuscany, we dine.
To make a dish of pasta so divine,
Follow this recipe, it's truly fine:

Take tomatoes ripe, as red as the rose,
In a pan, let them simmer and compose.
Add garlic, basil, and a pinch of spice,
Stir gently, like a lover's soft caress.

Boil pasta al dente, just so,
Toss it with the sauce, a perfect flow.
Grate Parmesan cheese, a snowy delight,
Sprinkle on top, a feast for the sight.

Serve with a glass of Chianti so grand,
In Italy's embrace, we joyously stand.
This simple dish, a taste of amore,
Lord Byron himself would surely adore.
